### Decision Trees

### DT are versatile ML algorithms that can perform regression and classification tsks

#### Advantages
* Easy to Understand
* Useful in Data exploration
* Less data cleaning required
* Data type is not a constraint

#### Disadvantages
* Over fitting

#### Objective: 
##### Build a model to predict "Drug Like" properties of a single compound.


#### Data: 
##### ADME descriptors for 3 libraries. Libraries: AFRODB Biofacquim FDA

#### Endpoint:
##### Drug Like (Binary)
* 1 -> Drug Like
*    0 -> No Drug Like

#### Descriptors:
#####    ADME descriptors:
* Aromatic heavy atoms
* H-bond acceptors
* H-bond donors
* Heavy atoms
* Rotatable bonds
* Ali Log S
* Ali Solubility (mg/ml)
#### Method: 
##### Decision Trees

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
Data = pd.read_csv("data/Data_SVM.csv", sep = ",")

In [ ]:
Data.head()

In [ ]:
## Exploratory Data Analysis

In [ ]:
#Identify Libraries
Data.Library.unique()

In [ ]:
#Identify Target
Data["Drug Like"].unique()

In [ ]:
#Plot descriptors
sns.boxplot(x = "Library", y = Data["MW"], data=Data)

In [ ]:
#Select numerical variables
numerical_data  = Data.select_dtypes(np.number)

In [ ]:
numerical_data.head(6)

In [ ]:
#statistical values
numerical_data.describe()

In [ ]:
#Correlation
corr = numerical_data.corr()
corr.head()

In [ ]:
sns.heatmap(corr)

In [ ]:
#save heatmap
plt.savefig("correlacion_inicial.png")

In [ ]:
#identify high correlated variables
corr_var =   ['XLOGP3', 'iLOGP', 'log Kp (cm/s)', 'Silicos-IT LogSw',
             'Ali Solubility (mol/l)','Ali Solubility (mg/ml)' , 
                'Consensus Log P', 'ESOL Solubility (mg/ml)', 'Unnamed: 54']

In [ ]:
#Drop correlated variables
numerical_data = numerical_data.drop(columns=corr_var)
#Drop Target variable
numerical_data = numerical_data.drop("Drug Like", axis =1)

In [ ]:
numerical_data.head()

In [ ]:
#Save target column in a new DF
df_target = pd.DataFrame(Data['Drug Like'],columns=['Drug Like'])
df_target['Drug Like'].unique()

In [ ]:
df_target

## Machine Learning Model

### Decision Tree

In [ ]:
#Import
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
#Create a test set
from sklearn.model_selection import train_test_split

In [ ]:
#split data
X_train, X_test, y_train, y_test = train_test_split(numerical_data, np.ravel(df_target)              , test_size=0.30, random_state=1992)

In [ ]:
X_train.head()

In [ ]:
y_train

In [ ]:
#Assign Model
dtree = DecisionTreeClassifier(criterion='gini', 
                               max_depth=4,
                               #max_leaf_nodes=8, 
                               random_state = 1992)


In [ ]:
#train model
dtree.fit(X_train, y_train)

### Tree visualization

In [ ]:
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot

In [ ]:
feature_names = numerical_data.columns
#feature_names
X_train.columns

In [ ]:
dot_data = StringIO()  
export_graphviz(dtree, out_file=dot_data,feature_names=feature_names,
                        filled=True,rounded=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
image_name = "DT.png"
graph[0].write_png(image_name)
display(Image(graph[0].create_png()))

Gini Index



### Predictions

In [ ]:
#write a function to select an specific compound
def test_compound(Data, Library, Name):
    Data = Data[Data["Library"]== Library]
    test = Data[Data["Name"]== Name]
    test = test[numerical_data.columns]
    #print(test.head())
    return test

In [ ]:
test = test_compound(Data, "FDA", "Acetaminophen")

In [ ]:
#Descriptors
test

#### single prediction

In [ ]:
dtree.predict(test)

### Evaluate the model

In [ ]:
#make predictions
predictions = dtree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
#accuracy
accuracy_score(y_test,predictions)

In [ ]:
print(confusion_matrix(y_test,predictions))

### Random Forests¶
#### Now let's compare the decision tree model to a random forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)


In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
test = test_compound(Data, "FDA", "Acetaminophen")
rfc.predict(test)

In [ ]:
rfc_pred = rfc.predict(X_test)

In [ ]:
rfc_pred[:10]


In [ ]:
print(confusion_matrix(y_test,rfc_pred))